In [17]:
import pandas as pd
import requests
from config import api_key
import json

In [18]:
rep_candidate=pd.read_csv("2020_rep_candidate.csv")
new_gop_finance=pd.read_csv("2020_gop_finance_data.csv")

In [19]:
rep_candidate=rep_candidate.rename(columns={
    "state":"candidate_state"
})
rep_finance=new_gop_finance.rename(columns={
    "state":"donor_state"
})



In [20]:
rep_data=pd.DataFrame.merge(rep_candidate, rep_finance, on="candidate_id")

In [21]:
rep_data=rep_data.rename(columns={
    "state":"donor_state"
})
rep_data

,Unnamed: 0_x,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,Unnamed: 0_y,donor_state,donation_count,total_raised,cycle
0,0,"ADAIR, STANLEY E",S0AL00248,AL,REP,Challenger,Senate,[2020],[2020],REP,C,0,AL,11,1428.99,2020
1,1,"ADKINS, GARY GENE",S0VA00096,VA,REP,Challenger,Senate,[2020],[2020],REP,C,1,VA,2,3200.00,2020
2,2,"ALEXANDER, LAMAR",S2TN00058,TN,REP,Incumbent,Senate,"[2002, 2004, 2006, 2008, 2010, 2012, 2014, 201...","[2002, 2008, 2014, 2020]",REP,C,2,VA,2,5000.00,2020
3,3,"AYYADURAI, SHIVA DR",S8MA00268,MA,REP,Challenger,Senate,"[2018, 2020]","[2018, 2020]",REP,C,3,AL,1,250.00,2020
4,3,"AYYADURAI, SHIVA DR",S8MA00268,MA,REP,Challenger,Senate,"[2018, 2020]","[2018, 2020]",REP,C,4,AR,1,500.00,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,18,"WHEELER, JOSHUA B",S0WY00103,WY,REP,Open seat,Senate,[2020],[2020],REP,C,1196,GA,1,500.00,2020
2257,18,"WHEELER, JOSHUA B",S0WY00103,WY,REP,Open seat,Senate,[2020],[2020],REP,C,1197,WY,7,5375.00,2020
2258,19,"WHITT, LARRY ALLEN II",S0WV00231,WV,REP,Challenger,Senate,[2020],[2020],REP,C,1198,AL,3,2250.00,2020
2259,19,"WHITT, LARRY ALLEN II",S0WV00231,WV,REP,Challenger,Senate,[2020],[2020],REP,C,1199,TX,3,1000.00,2020


In [23]:
# create data frame with totals fundraised by every candidate in every state they fundraised

rep_data=rep_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
rep_data.shape

(2261, 12)

In [24]:
# if candidate_state = donor_state, in_state=True, if not, then False

in_state=(rep_data.candidate_state==rep_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state

,in_state
0,True
1,True
2,False
3,False
4,False
...,...
2256,False
2257,True
2258,False
2259,False


In [25]:
new_rep_data=pd.DataFrame.merge(rep_data,in_state, left_index=True, right_index=True)

new_rep_data.set_index("candidate_id")
new_rep_data.shape

(2261, 13)

In [29]:
new_rep_data["candidate_id"].nunique()
rep_aggregate=new_rep_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})
rep_aggregate


rep_aggregate.columns = rep_aggregate.columns.droplevel(1)

rep_aggregate=rep_aggregate.reset_index()

in_state=rep_aggregate.loc[rep_aggregate["in_state"]==True]
in_state
out_of_state=rep_aggregate.loc[rep_aggregate["in_state"]==False]
out_of_state

state_agg=pd.DataFrame.merge(in_state, out_of_state, on="candidate_id", suffixes=("_in_state","_out_of_state"), how="outer")
# state_agg.to_csv("rep_finance_data.csv")

state_agg.shape
rep_candidates=pd.read_csv("2020_rep_candidate.csv")
final_rep_data=pd.DataFrame.merge(rep_candidates, state_agg, on="candidate_id")
final_rep_data.shape

(102, 17)

In [30]:
final_rep_data_2020=final_rep_data.filter(["name","candidate_id","state", "party","incumbent_challenge_full","cycles","donation_count_in_state","total_raised_in_state","donation_count_out_of_state","total_raised_out_of_state"])

In [32]:
final_rep_data.to_csv("2020_final_rep_data.csv")